# Derma-Challenge

Para  comenzar, importaremos las bibliotecas necesarias

In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

## Data Exploration

- Cargar y visualizar algunas imágenes de muestra
- Analizar la distribución de clases
- Calcular estadísticas básicas de las imágenes

In [2]:
data_path = "images/images"

out_data_path = "images/data"

In [ ]:
if os.path.exists(out_data_path):
    os.mkdir(out_data_path)
    os.mkdir(os.path.join(out_data_path, "train"))
    os.mkdir(os.path.join(out_data_path, "test"))
    os.mkdir(os.path.join(out_data_path, "val"))

## Data Preprocessing

- Aplicar transformaciones a las imágenes (redimensionamiento, normalización, etc.)
- Dividir el conjunto de datos en entrenamiento, validación y prueba
- Crear los DataLoaders de PyTorch

### Transformaciones

In [ ]:
# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize(256),  # Redimensionar a 256x256
    transforms.CenterCrop(224),  # Recortar el centro de la imagen a 224x224
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizar
])

In [ ]:
# Cargar una imagen de ejemplo
img = Image.open('imagen_ejemplo.jpg')
img

In [ ]:
# Aplicar las transformaciones
img_transformada = transform(img)
img_transformada

In [ ]:
# Mostrar la imagen transformada
plt.imshow(img_transformada.permute(1, 2, 0))  # Permutar los canales para que sea compatible con matplotlib
plt.show()

### DataLoaders

In [ ]:
# Definir la clase Dataset para cargar las imágenes y etiquetas
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        label = int(img_name.split('.')[0].split('_')[-1])  # Asignar etiqueta según el nombre del archivo
        return image, label



In [ ]:
# Crear el conjunto de datos
dataset = ImageDataset('path/to/dataset', transform=transform)
dataset

In [ ]:

# Crear el DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Probar el DataLoader
for batch in dataloader:
    images, labels = batch
    print(images.shape, labels.shape)
    break

## Model Training 

- Definir la arquitectura del modelo
- Configurar el optimizador y la función de pérdida
- Entrenar el modelo y monitorear el progreso
- Guardar el modelo entrenado

## Model Evaluation

- Cargar el modelo entrenado
- Evaluar el modelo en el conjunto de prueba
- Calcular métricas de rendimiento
- Visualizar ejemplos de predicciones correctas e incorrectas